In [1]:
import os
import pandas as pd

In [2]:
id_mapping = pd.read_csv("ID_mapping.csv", dtype = "str")
id_mapping["Long COVID BUcket ID"]

0       gxezl15259939791237511
1       gxezl12611565318297434
2       gxezl11789746375185174
3       gxezl16295349979974429
4       gxezl16269652497371226
                 ...          
5568    gxezl16978192592929921
5569    gxezl12543319137957218
5570    gxezl13681398624153695
5571    gxezl15612257367486858
5572    gxezl11274228981612658
Name: Long COVID BUcket ID, Length: 5573, dtype: object

In [6]:
import os
import pandas as pd

id_mapping = pd.read_csv("ID_mapping.csv", dtype = "str")

f = open("/labs/mpsnyder/long-covid-study-data/final_data/output2.txt",'w')
invalid_files = []
gcb_path = [
    "/labs/mpsnyder/long-covid-study-data/COVID_GCB/covid-study/",
    "/labs/mpsnyder/long-covid-study-data/longcovid2022-study/longcovid2022-processed/"
]
scg_folder_path = [
    "/labs/mpsnyder/long-covid-study-data/LongCOVIDSCG/All_others_wearable_data_Mar2022/", 
    "/labs/mpsnyder/long-covid-study-data/LongCOVIDSCG/All_positives_wearable_data_Mar2022/",
    "/labs/mpsnyder/long-covid-study-data/LongCOVIDSCG/Older/"
]
data_formats = {
    "hr": ["/hr/hr.csv","/hr.csv", "/Orig_Fitbit_HR.csv", "/Orig_NonFitbit_HR.csv"],
    "st": ["/st/st.csv","/st.csv", "/Orig_Fitbit_ST.csv", "/Orig_NonFitbit_ST.csv"],
    "sl": ["/sl/sl.csv","/sl.csv"],
    "heartvar": ["/heartvar.csv"],
    "resheart": ["/resheart.csv"],
    "userAlertdata": ["/userAlertdata.csv"],
    "vo2": ["/vo2.csv"],
}
# change this count to start from any row of the id_mapping
count = 750
for new_id in id_mapping["Long COVID BUcket ID"].iloc[count:1000]:
    print("start " + str(count), file=f)
    print("start " + str(count))
    f.flush()
    count += 1
    # loop through all the different types of data
    for data_format, data_paths in data_formats.items():
        dfs = []
        # loop through folders for long-covid-study and covid-study
        for path in gcb_path:
            if os.path.isdir(path + new_id):
                for data_path in data_paths:
                    if os.path.isfile(path + new_id + data_path) and os.stat(path + new_id + data_path).st_size != 0:
                        print(path + new_id + data_path, file=f)
                        print(path + new_id + data_path)
                        try:
                            dfs.append(pd.read_csv(path + new_id + data_path, dtype = "string"))
                        except Exception as e:
                            print("ERROR", file=f)
                            print(e, file=f)
                            print(e)
                            continue
        
        # loop through scg folders
        for path in scg_folder_path:
            if os.path.isdir(path + new_id):
                for data_path in data_paths:
                    if os.path.isfile(path + new_id + data_path) and os.stat(path + new_id + data_path).st_size != 0:
                        try:
                            df_scg_raw = pd.read_csv(path + new_id + data_path, dtype = "string")
                        except Exception as e:
                            print("ERROR", file=f)
                            print(e, file=f)
                            print(e)
                            continue
                        # add device column
                        if data_path == "/Orig_Fitbit_HR.csv" or data_path == "/Orig_Fitbit_ST.csv":
                            df_scg_raw["device"] = "Fitbit"
                        # remap all the columns
                        df_scg_raw = df_scg_raw.rename(columns={"device": "Device", "heartrate": "Value", "steps": "Value"})
                        if "datetime" in df_scg_raw:
                            df_scg_raw[['Start_Date','Start_Time']] = df_scg_raw.datetime.str.split(n=1, expand=True)
                        else:
                            invalid_files += path + new_id + data_path
                            continue
                        print(path + new_id + data_path, file=f)
                        print(path + new_id + data_path)
                        dfs.append(df_scg_raw)
        if len(dfs) == 0: continue
        df_concat = pd.concat(dfs, axis=0, ignore_index=True)
        if len(df_concat.index) == 0:
            df_final.to_csv("/labs/mpsnyder/long-covid-study-data/final_data/" + new_id + "/" + data_format + ".csv", index = False)
            continue
        if "Start_Time" in df_concat:
            if df_concat.iloc[0]['Start_Time'].endswith("M"):
                df_concat["Start_Time"] = pd.to_datetime(df_concat["Start_Time"])
            df_final = df_concat.sort_values(by=["Start_Date", "Start_Time"], ascending=[True, True], ignore_index = True, na_position = "first")
        elif "Start_Date" in df_concat:
            df_final = df_concat.sort_values(by=["Start_Date"], ascending=[True], ignore_index = True, na_position = "first")
        else:
            invalid_files += new_id + data_path
        # df_concat
        df_final = df_final.drop(['Unnamed: 0.1','Unnamed: 0', "datetime"], axis=1, errors='ignore')
        df_final.to_csv("/labs/mpsnyder/long-covid-study-data/final_data/" + new_id + "/" + data_format + ".csv", index = False)

start 750
start 751
start 752
/labs/mpsnyder/long-covid-study-data/LongCOVIDSCG/All_others_wearable_data_Mar2022/gxezl12293338164517632/Orig_Fitbit_HR.csv
/labs/mpsnyder/long-covid-study-data/COVID_GCB/covid-study/gxezl12293338164517632/st/st.csv
/labs/mpsnyder/long-covid-study-data/LongCOVIDSCG/All_others_wearable_data_Mar2022/gxezl12293338164517632/Orig_Fitbit_ST.csv
start 753
start 754
start 755
/labs/mpsnyder/long-covid-study-data/LongCOVIDSCG/All_others_wearable_data_Mar2022/gxezl11981494351173486/Orig_Fitbit_HR.csv
/labs/mpsnyder/long-covid-study-data/COVID_GCB/covid-study/gxezl11981494351173486/st/st.csv
/labs/mpsnyder/long-covid-study-data/LongCOVIDSCG/All_others_wearable_data_Mar2022/gxezl11981494351173486/Orig_Fitbit_ST.csv
/labs/mpsnyder/long-covid-study-data/COVID_GCB/covid-study/gxezl11981494351173486/sl/sl.csv
start 756
/labs/mpsnyder/long-covid-study-data/COVID_GCB/covid-study/gxezl15183952313153299/hr/hr.csv
/labs/mpsnyder/long-covid-study-data/LongCOVIDSCG/All_others_w

KeyboardInterrupt: 